In [1]:
Capital = 10000  # Первоначальный капитал
t = 22*13 # Количество временных интервалов для расчетов (22*13 - это 1 месяц). То есть в данном случае алгоритм сичтает, 
          # что заранее известны только данные за месяц, на основе их анализа он принимает решения. 
          # Данные за остальное время изначально считаются неизвестными, на них демонстрируется работоспособность алгоритма.
T = 100  #Период, по которому аппроксимируется спред
tax = 0.0005 # Доля которую удерживает брокер при сделке
k = 0.9
t_start = 20201001
t_finish = 20211101

In [2]:
import os
import numpy as np
import csv
import pandas as pd
import matplotlib.pyplot as plt

data_start = np.zeros((2000, 11000))
name_share = dict()

for i, filename in enumerate(os.listdir("/content/drive/MyDrive/МосБиржа топ_01_04_2020_01_04_2022")):
   with open(os.path.join("/content/drive/MyDrive/МосБиржа топ_01_04_2020_01_04_2022", filename), 'r') as f:
      reader = csv.reader(f, delimiter = ",")
      for j, row in enumerate(reader):
        if j == 0:
          data_start[i+1][j] = i+1
        else:
          data_start[i+1][j] = float(row[4])
        if i == 0 and j != 0:   
          data_start[i][j] = int(row[2])
        name_share[i+1] = filename

print(name_share)

{1: 'polymetal_poly_1_1_moex-akcii_polymetal-international-plc.csv', 2: 'yandex cla_yndx_1_1_moex-akcii_pllc-yandex-n-v.csv', 3: 'алроса ао_alrs_2_1_moex-akcii_alrosa-ao.csv', 4: 'акрон_akrn_0_1_moex-akcii_akron.csv', 5: 'аэрофлот_aflt_2_1_moex-akcii_aeroflot.csv', 6: 'бсп ао_bspb_2_1_moex-akcii_bsp.csv', 7: 'башнефт ао_bane_1_1_moex-akcii_bashneft-ank-ao.csv', 8: 'башнефт ап_banep_1_1_moex-akcii_bashneft-ank-ap.csv', 9: 'всмпо-авсм_vsmo_0_1_moex-akcii_verhnesald-mpo.csv', 10: 'втб ао_vtbr_6_1_moex-akcii_vtb.csv', 11: 'газпром ао_gazp_2_1_moex-akcii_gazprom.csv', 12: 'гмкнорник_gmkn_0_1_moex-akcii_nornickel-gmk.csv', 14: 'интерраоао_irao_4_1_moex-akcii_inter-rao-ao.csv', 15: 'камаз_kmaz_1_1_moex-akcii_kamaz.csv', 16: 'лср ао_lsrg_1_1_moex-akcii_lsr.csv', 17: 'лукойл_lkoh_1_1_moex-akcii_lukoil.csv', 18: 'м.видео_mvid_1_1_moex-akcii_m-video.csv', 19: 'ммк_magn_3_1_moex-akcii_mmk.csv', 20: 'мтс-ао_mtss_2_1_moex-akcii_mts.csv', 21: 'магнит ао_mgnt_1_1_moex-akcii_magnit.csv', 23: 'мосбиржа

In [3]:
data_dt = data_start[:, (data_start[0,:] >= t_start) & (data_start[0,:] <= t_finish)].copy()
non_zero_data = np.any(data_dt != 0, axis = 1)

data = np.column_stack((data_start[:,0], data_dt))
data = data[non_zero_data, :].copy()



[[0.0000000e+00 2.0201001e+07 2.0201001e+07 ... 2.0211101e+07
  2.0211101e+07 2.0211101e+07]
 [1.0000000e+00 1.6900000e+03 1.6947000e+03 ... 1.3313000e+03
  1.3310000e+03 1.3298000e+03]
 [2.0000000e+00 5.0686000e+03 5.0724000e+03 ... 5.8816000e+03
  5.8836000e+03 5.8974000e+03]
 ...
 [4.7000000e+01 2.8500000e+03 2.8380000e+03 ... 5.7950000e+03
  5.7990000e+03 5.7980000e+03]
 [4.8000000e+01 1.9275000e+03 1.9305000e+03 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]
 [4.9000000e+01 2.7210000e+00 2.7100000e+00 ... 0.0000000e+00
  0.0000000e+00 0.0000000e+00]]


In [4]:
def corel(first_share, second_share):
  Delta_GR = 0
  Delta_G2 = 0
  Delta_R2 = 0
  mean_first_share = sum(first_share)/len(first_share)
  mean_second_share = sum(second_share)/len(second_share)

  for i in range(len(first_share)):
      Delta_GR = Delta_GR + (first_share[i]- mean_first_share)*(second_share[i]- mean_second_share)
      Delta_G2 = Delta_G2 + (first_share[i]- mean_first_share)**2
      Delta_R2 = Delta_R2 + (second_share[i]- mean_second_share)**2

  cor = Delta_GR/((Delta_G2*Delta_R2)**0.5)
  return (cor)

In [5]:
# Метод наименьших квадратов. Полином пятой степени.

def MNK(X, Y, max_pow=8):
  A = np.vstack([np.array(X)**i for i in range(max_pow+1)]).T
  Norm = np.linalg.pinv(A).dot(Y)
  return A @ Norm

In [6]:
def main_prib(first_share, second_share, time, t, tax):
  first_share_norm = MNK(first_share[0:t], second_share[0:t], 1)
  ind = 0
  Sdel = []
  N_r, N_g = 0, 0
  Prib = Capital
  Stoim = Capital
  First_price = 0
  Second_price = 0
  Activ_share = 0
  if second_share[t-1]-first_share_norm[t-1] < 0:
    ind = 1
  for i in range(t,len(second_share)):
      first_share_norm = MNK(first_share[i-t:i], second_share[i-t:i], 1)
      Spred = second_share[i-T:i] - first_share_norm[t-T:t]
      Spred_approx = MNK(time[i-T:i], Spred[0:T], 5)
      if second_share[i] > Second_price + 2*Stoim*tax // first_share[i]\
        and Spred_approx[len(Spred_approx)-2] > Spred_approx[len(Spred_approx)-1]  \
        and Spred[T-1] >0 and ind == 0 and abs(Spred[T-1]) > (2*Stoim // first_share[i])*5*tax:
          Prib = Prib + N_r*second_share[i]*(1-tax)
          #print(Axis[i],'Продали', round(N_r),'шт. привелигированных по цене',second_share[i],'стоимость портфеля =', Prib )
          N_r = 0
          N_g = Prib//(first_share[i]*(1+tax))
          Prib = Prib - N_g*first_share[i]*(1+tax)
          Stoim = Prib + N_g*first_share[i]*(1-tax)
          First_price = first_share[i]
          ind = 1
          Activ_share = 1  # Совершена сделка с арбитражом
          #print(Axis[i],'Купили ',round(N_g),'шт. обычных акций по цене',first_share[i],', остаток = ',Prib,'стоимость портфеля =', Stoim)
      elif first_share[i] > First_price + 2*Stoim*tax//second_share[i] \
        and Spred_approx[len(Spred_approx)-2] < Spred_approx[len(Spred_approx)-1] \
        and Spred[T-1]<0 and ind == 1 and abs(Spred[T-1]) > (2*Stoim//second_share[i])*5*tax:
          Prib = Prib + N_g*first_share[i]*(1-tax)
          #print(Axis[i],'Продали ',round(N_g),'шт. обычных по цене',first_share[i],'стоимость портфеля =', Prib)
          N_g = 0
          N_r = Prib//second_share[i]*(1+tax)
          Prib = Prib - N_r*second_share[i]*(1+tax)
          Stoim = Prib + N_r*second_share[i]*(1-tax)
          Second_price = second_share[i]
          ind = 0
          Activ_share = 1  # Совершена сделка с арбитражом
          #print(Axis[i],'Купили ',round(N_r),'шт. привелигированных по цене',second_share[i],', остаток = ',Prib,'стоимость портфеля =', Stoim)
  return (Stoim, Activ_share)

In [7]:
from tqdm import tqdm
import statistics

In [8]:
Final_stoim = []
Prib_null = 0
D_prib = []

for i in tqdm(range(1,len(data))):
  Prib_null_first = 0
  Prib_null_second = 0
  Dobavochnay_prib = 0
  for j in range(i+1,len(data)):
    Kor_d = corel(data[i][:t], data[j][:t])
    if Kor_d > k and Kor_d != 1:
      S, Activ_share = main_prib(data[i], data[j], np.arange(0,data.shape[1]), t, tax)
      if Activ_share == 1:
        Prib_final = (S - Capital)/Capital*100
        Final_stoim.append(Prib_final)
        Prib_null_first = (data[i,len(data[i])-1]- data[i,t])/data[i,t]*100
        Prib_null_second = (data[j,len(data[j])-1]- data[j,t])/data[j,t]*100
        Prib_null = Prib_null + Prib_null_second + Prib_null_first
        Dobavochnay_prib = ((S - Capital)/Capital*100) - ((Prib_null_first+Prib_null_second)/2)
        #print("Первая акция : ", name_share[data[i][0]], "Доходность без арбитража : ", Prib_null_first)
        #print("Вторая акция : ", name_share[data[j][0]], "Доходность без арбитража : ", Prib_null_second)
        #print("С арбитражом : ", (S - Capital)/Capital*100)
        #print("Добавочная прибыль : ", Dobavochnay_prib)
        D_prib.append(Dobavochnay_prib)

Count_minus = 0
Count_ten = 0

for i in range(len(Final_stoim)):
    if Final_stoim[i] < 0:
      Count_minus = Count_minus + 1
    if Final_stoim[i] < 10:
      Count_ten = Count_ten + 1


print("Средняя добавочная доходность :", statistics.mean(D_prib))
print("Средняя доходность : ", statistics.mean(Final_stoim))
print("Медианное значение : ", statistics.median(Final_stoim))
print("Минимальное значение : ", min(Final_stoim))
print("Максимальное значение : ", max(Final_stoim))
print("Количество пар с отрицательной доходность : ", Count_minus)
print("Количество пар с доходностью < 10% : ", Count_ten)

100%|██████████| 45/45 [02:05<00:00,  2.80s/it]

Средняя добавочная доходность : 41.399306209478034
Средняя доходность :  31.316207799359333
Медианное значение :  19.207456675625146
Минимальное значение :  -0.09883939499999543
Максимальное значение :  136.21159931900036
Количество пар с отрицательной доходность :  11
Количество пар с доходностью < 10% :  53


In [9]:
print(((data[1,t] - data[1,len(data[1])-1])/data[1,t]*100))
print(Prib_null)
len(Final_stoim)

22.279368790181184
-2903.9323421141853


144

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
